# import

In [ ]:
!pip3 install swifter

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Load

In [7]:
import sys
sys.path.append('/content/drive/MyDrive/MemeProject/src/pytorch/augmentation')
print(sys.path)

['/content', '/env/python', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.9/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/MemeProject/src/pytorch/augmentation']


In [8]:
import swifter
import config as cfg
from pprint import pprint
from pshmodule.utils import filemanager as fm

In [9]:
df = fm.load('drive/MyDrive/MemeProject/data/augmentation/aug_51007.xlsx')

extension : .xlsx
Loaded 52367 records from drive/MyDrive/MemeProject/data/augmentation/aug_51007.xlsx


In [10]:
# 불필요 제거
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df = df.iloc[1:]

# 정렬
df.sort_values(['g_num', 'u_num'], ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)

# int→str
df = df.astype({'g_num':'str', 'u_num':'str', 'label':'str'})
df['g_num'] = df.g_num.apply(lambda x: x.replace('.0', ''))
df['u_num'] = df.u_num.apply(lambda x: x.replace('.0', ''))
df['label'] = df.label.apply(lambda x: x.replace('.0', ''))

<ipython-input-10-dfaa772b3f0a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(['g_num', 'u_num'], ascending=True, inplace=True)


In [11]:
user_dict = fm.load(cfg.user_dict)

extension : .xlsx
Loaded 5103 records from /content/drive/MyDrive/MemeProject/data/user_dict/UGCD.xlsx


/usr/local/lib/python3.9/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Dict

In [12]:
user_dict_2 = user_dict[5:]
user_dict_2.columns = user_dict.iloc[5]
user_dict_2.reset_index(drop=True, inplace=True)

In [13]:
user_dict_2 = user_dict_2[1:]
user_dict_2 = user_dict_2[['표제어', '기본형', '품사', '감정1', '연령', '용례', '반의어']]
user_dict_2.rename(columns={'표제어':'lemma', '기본형':'formal', '품사':'pos', '감정1':'sentiment', '연령':'age', '용례':'example', '반의어':'antonym'}, inplace=True)
user_dict_2.reset_index(drop=True, inplace=True)

In [14]:
user_dict_2.head(10)

5,lemma,formal,pos,sentiment,age,example,antonym
0,가지다,가지다,VV,Neu,NaN,그거 내가 가진다,버리다
1,갖다,가지다,VV,Neu,NaN,이 지갑은 내가 갖는다,버리다
2,내 거 하다,가지다,VV,Hap,MZ,걔 내 거 할 거야,NaN
3,소유하다,가지다,VV,Hap,NaN,그 신발은 이미 소유했어,버리다
4,보유하다,가지다,VV,Hap,NaN,나도 그 아이템 보유하고 싶어,버리다
5,겟하다,가지다,VV,Hap,MZ,드디어 겟했다,버리다
6,겟또하다,가지다,VV,Hap,MZ,피규어 겟또했음,버리다
7,득템하다,가지다,VV,Hap,MZ,신상템 득템했다,버리다
8,굴러들어 오다,가지다,VV,Hap,NaN,꽁돈이 굴러들어 왔음!,NaN
9,손에 넣다,가지다,VV,Hap,NaN,한정판 운동화 손에 넣었다,NaN


In [15]:
user_dict_2.shape

(5097, 7)

In [16]:
user_dict_3 = user_dict_2[~user_dict_2.lemma.isnull()]

In [17]:
user_dict_3.shape

(3315, 7)

In [18]:
user_dict_3.pos.value_counts()

VV     1398
NN     1234
VA      308
MAG     154
NNP      82
IC       74
EO       46
NP       17
MM        2
Name: pos, dtype: int64

Tag Mapping

Okt

In [19]:
def okt(pos):
  result = ""
  if pos in ['NN', 'NNP', 'NR', 'NP']:
    result = "Noun" # 명사
  elif pos == "VV":
    result = "Verb" # 동사
  elif pos == "VA":
    result = "Adjective" # 형용사
  elif pos == "MM":
    result = "Determiner" # 관형사
  elif pos in ['MAG', 'MAJ']:
    result = "Adverb" # 부사
  elif pos == "IC":
    result = "Exclamation" # 감탄사
  elif pos == "JO":
    result = "Josa" # 조사
  elif pos == "EO":
    result = "Eomi" # 어미
  return result

In [20]:
user_dict_3['okt_pos'] = user_dict_3.pos.swifter.apply(okt)

Pandas Apply:   0%|          | 0/3315 [00:00<?, ?it/s]

<ipython-input-20-780ddc5911d3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_dict_3['okt_pos'] = user_dict_3.pos.swifter.apply(okt)


In [21]:
user_dict_3.head()

5,lemma,formal,pos,sentiment,age,example,antonym,okt_pos
0,가지다,가지다,VV,Neu,NaN,그거 내가 가진다,버리다,Verb
1,갖다,가지다,VV,Neu,NaN,이 지갑은 내가 갖는다,버리다,Verb
2,내 거 하다,가지다,VV,Hap,MZ,걔 내 거 할 거야,NaN,Verb
3,소유하다,가지다,VV,Hap,NaN,그 신발은 이미 소유했어,버리다,Verb
4,보유하다,가지다,VV,Hap,NaN,나도 그 아이템 보유하고 싶어,버리다,Verb


Mecab

In [22]:
def mecab(pos):
  result = ""
  if pos == "NN":
    result = "NNG" # 일반 명사
  elif pos == "NNP":
    result = "NNP" # 고유 명사
  elif pos == "NR":
    result = "NR" # 수사
  elif pos == "NP":
    result = "NP" # 대명사
  elif pos == "VV":
    result = "VV" # 동사
  elif pos == "VA":
    result = "VA" # 형용사
  elif pos == "MM":
    result = "MM" # 관형사
  elif pos == "MAG":
    result = "MAG" # 일반부사
  elif pos == "MAJ":
    result = "MAJ" # 접속부사
  elif pos == "IC":
    result = "IC" # 감탄사
  elif pos == "JO":
    result = "JKS" # 조사
  return result

In [23]:
user_dict_3['mecab_pos'] = user_dict_3.pos.swifter.apply(mecab)

Pandas Apply:   0%|          | 0/3315 [00:00<?, ?it/s]

<ipython-input-23-ed5046f81635>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_dict_3['mecab_pos'] = user_dict_3.pos.swifter.apply(mecab)


# Add Dict

### Okt

test

In [ ]:
import konlpy
konlpy.data.path

['/root/konlpy_data',
 '/usr/share/konlpy_data',
 '/usr/local/share/konlpy_data',
 '/usr/lib/konlpy_data',
 '/usr/local/lib/konlpy_data',
 '/usr/local/lib/python3.8/dist-packages/konlpy/data']

In [ ]:
from konlpy.tag import Okt

In [ ]:
okt = Okt()

In [ ]:
user_dict_3.pos.value_counts()

NameError: ignored

In [ ]:
user_dict_3[user_dict_3.pos=='VA']

5,lemma,formal,pos,sentiment,age,example,antonym,okt_pos,mecab_pos
30,귀엽다,귀엽다,VA,Hap,NaN,나 귀여워,NaN,Adjective,VA
31,쏘 러블리하다,귀엽다,VA,Hap,MZ,너 쏘 러블리해,NaN,Adjective,VA
32,존귀탱이다,귀엽다,VA,Hap,MZ,님 존귀탱임,NaN,Adjective,VA
33,쏘 큐트하다,귀엽다,VA,Hap,MZ,얘 쏘 큐트하다,NaN,Adjective,VA
34,초 카와이하다,귀엽다,VA,Hap,MZ,이거 초 카와이해,NaN,Adjective,VA
...,...,...,...,...,...,...,...,...,...
3241,눈물버튼이다,슬프다,VA,Sad,MZ,이거 내 눈물버튼인데.,NaN,Adjective,VA
3242,비상이다,슬프다,VA,Sad,MZ,비상이다... 비상...,NaN,Adjective,VA
3265,꾸꾸꾸,꾸꾸꾸,VA,Dis,MZ,뭘 해도 꾸꾸꾸다,NaN,Adjective,VA
3266,꾸며도 꾸질 꾸질하다,꾸꾸꾸,VA,Dis,MZ,내 주변에 꾸며도 꾸질 꾸질한 사람 있음ㅋ,NaN,Adjective,VA


In [ ]:
print(okt.pos('핸드폰을 겟또하다.'))
print(okt.pos('동전을 득템하다.'))
print(okt.pos('가상화폐를 살거야'))
print(okt.pos('틀니딱딱 거리지마~'))
print(okt.pos('엄마가 젤루 좋아'))
print(okt.pos('이번엔 틀림없이 합격할 거야'))
print(okt.pos('쏘 러블리하다'))

[('핸드폰', 'Noun'), ('을', 'Josa'), ('겟또하다', 'Verb'), ('.', 'Punctuation')]
[('동전', 'Noun'), ('을', 'Josa'), ('득템하다', 'Verb'), ('.', 'Punctuation')]
[('가상화폐', 'Noun'), ('를', 'Josa'), ('살거야', 'Verb')]
[('틀니딱딱', 'Noun'), ('거리', 'Noun'), ('지', 'Josa'), ('마', 'Noun'), ('~', 'Punctuation')]
[('엄마', 'Noun'), ('가', 'Josa'), ('젤루', 'Adverb'), ('좋아', 'Adjective')]
[('이번', 'Noun'), ('엔', 'Josa'), ('틀림없이', 'Adverb'), ('합격', 'Noun'), ('할', 'Verb'), ('거야', 'Eomi')]
[('쏘', 'Verb'), ('러블리하다', 'Adjective')]


add dict

In [ ]:
import os

os.chdir('/usr/local/lib/python3.8/dist-packages/konlpy/java')
os.getcwd()

'/usr/local/lib/python3.8/dist-packages/konlpy/java'

In [ ]:
!ls 

aho-corasick.jar     kkma-2.0.jar		 shineware-ds-1.0.jar
bin		     komoran-3.0.jar		 snakeyaml-1.12.jar
conf		     open-korean-text-2.1.0.jar  twitter-text-1.14.7.jar
data		     scala-library-2.12.3.jar
jhannanum-0.8.4.jar  shineware-common-1.0.jar


In [ ]:
!jar xvf open-korean-text-2.1.0.jar

In [ ]:
user_dict_3.okt_pos.value_counts()

Verb           1398
Noun           1333
Adjective       308
Adverb          154
Exclamation      74
Eomi             46
Determiner        2
Name: okt_pos, dtype: int64

In [ ]:
paths = {
    'noun/nouns.txt':'Noun',
    'verb/verb.txt':'Verb',
    'adjective/adjective.txt':'Adjective',
    'adverb/adverb.txt':'Adverb',
    'josa/josa.txt':'Josa',
    'auxiliary/exclamation.txt':'Exclamation',
    'verb/eomi.txt':'Eomi',
    'auxiliary/determiner.txt':'Determiner'
}

In [ ]:
for p in paths:
  with open(f"/usr/local/lib/python3.8/dist-packages/konlpy/java/org/openkoreantext/processor/util/{p}") as f:
    data = f.read()
  
  data += '\n'.join(map(str, user_dict_3[user_dict_3.okt_pos == paths[p]].lemma.tolist())) + '\n'
  
  with open(f"/usr/local/lib/python3.8/dist-packages/konlpy/java/org/openkoreantext/processor/util/{p}", 'w') as f:
    f.write(data)
  data = ""

In [ ]:
!ls /usr/local/lib/python3.8/dist-packages/konlpy/java/org/openkoreantext/processor/util

ls: cannot access '/usr/local/lib/python3.8/dist-packages/konlpy/java/org/openkoreantext/processor/util': No such file or directory


In [ ]:
with open(f"/usr/local/lib/python3.8/dist-packages/konlpy/java/org/openkoreantext/processor/util/adjective/adjective.txt") as f:
  data = f.read()

FileNotFoundError: ignored

In [ ]:
!ls /usr/local/lib/python3.8/dist-packages/konlpy/java

aho-corasick.jar     kkma-2.0.jar		 scala-library-2.12.3.jar
bin		     komoran-3.0.jar		 shineware-common-1.0.jar
conf		     META-INF			 shineware-ds-1.0.jar
data		     open-korean-text-2.1.0.jar  snakeyaml-1.12.jar
jhannanum-0.8.4.jar  org			 twitter-text-1.14.7.jar


In [ ]:
!rm open-korean-text-2.1.0.jar
!jar cvf open-korean-text-2.1.0.jar org
!rm -rf org

check

In [ ]:
from konlpy.tag import Okt

In [ ]:
okt = Okt()

In [ ]:
print(f"겟또하다 Verb : {okt.pos('핸드폰을 겟또하다.')}")
print(f"득템하다 Verb : {okt.pos('동전을 득템하다.')}")
print(f"가상화폐 Noun : {okt.pos('가상화폐를 살거야')}")
print(f"틀니딱딱 Noun : {okt.pos('틀니딱딱 거리지마~')}")
print(f"젤루 Adverb : {okt.pos('엄마가 젤루 좋아')}")
print(f"틀림없이 Adverb : {okt.pos('이번엔 틀림없이 합격할 거야')}")
print(f"쏘 러블리하다 Adjective : {okt.pos('쏘 러블리하다')}")
print(f"가죽 무늬 Noun : {okt.pos('가죽 무늬 지갑을 샀어!')}")
print(f"러블리 그 잡채다 Verb : {okt.pos('러블리 그 잡채다')}")

겟또하다 Verb : [('핸드폰', 'Noun'), ('을', 'Josa'), ('겟또하다', 'Verb'), ('.', 'Punctuation')]
득템하다 Verb : [('동전', 'Noun'), ('을', 'Josa'), ('득템하다', 'Verb'), ('.', 'Punctuation')]
가상화폐 Noun : [('가상화폐', 'Noun'), ('를', 'Josa'), ('살거야', 'Verb')]
틀니딱딱 Noun : [('틀니딱딱', 'Noun'), ('거리', 'Noun'), ('지', 'Josa'), ('마', 'Noun'), ('~', 'Punctuation')]
젤루 Adverb : [('엄마', 'Noun'), ('가', 'Josa'), ('젤루', 'Adverb'), ('좋아', 'Adjective')]
틀림없이 Adverb : [('이번', 'Noun'), ('엔', 'Josa'), ('틀림없이', 'Adverb'), ('합격', 'Noun'), ('할', 'Verb'), ('거야', 'Eomi')]
쏘 러블리하다 Adjective : [('쏘', 'Verb'), ('러블리하다', 'Adjective')]
가죽 무늬 Noun : [('가죽', 'Noun'), ('무늬', 'Noun'), ('지갑', 'Noun'), ('을', 'Josa'), ('샀어', 'Verb'), ('!', 'Punctuation')]
러블리 그 잡채다 Verb : [('러블리', 'Noun'), ('그', 'Noun'), ('잡채', 'Noun'), ('다', 'Josa')]


### Kiwi

##### Install

In [ ]:
!pip install kiwipiepy

In [25]:
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords

from tqdm import tqdm
import pandas as pd

In [26]:
kiwi = Kiwi()
stopwords = Stopwords()

##### UserDict Add

1. 일단 사용자 사전을 공백을 제거 후 등록(불변어 기준)
2. 토크나이징 할 때 어떤 문장이 들어오면 공백 제거 후 토큰화

In [27]:
constant = ['NNG', 'NNP', 'NR', 'NP', 'MM', 'MAG', 'MAJ', 'IC']

In [28]:
# 합성어 기준
for c in constant:
  for i in user_dict_3[user_dict_3.mecab_pos==c].iterrows():
    kiwi.add_user_word(str(i[1]['lemma']).replace(' ', ''), c, 0)

##### Augmentation

1. 입력 문과 사용자 사전 표제어를 하은 님이 추출해준 실질 형태소&불변어 들만 살린 후 사용자 사전 실질 형태소가 입력 문 실질 형태소에 있는지를 비교함.<br>
2. 포함이 있으면 유의어 셋으로 교체하는 방법 - 가변어, 사용자 사전 추가 불변어는 공백 제거로 비교 후 교체(키 밸류로 원본 들고 있다가 교체 후 뱉기? 확인)<br>
3. 테스트 : 품사 별로 용례가 입력문으로 가정해서 표제어랑 비교해서 위 상황이 비교될만 한지

* 동사, 형용사 등 가변어는 형태소 분석 과정에서 어근 추출을 잘 못하거나 품사를 다르게 추출하는 경우가 생겨서 좀 더 고민이 필요해 보이고 따라서 불변어 위주로 교체가 우선으로 되려고 함.

평균 미만

In [29]:
count = {i[1]['label']: 0 for i in df.iterrows()}
for i in df.iterrows():
  count[i[1]['label']] += 1
sorted_dict = dict(sorted(count.items(), key = lambda item: item[1], reverse = True))

In [30]:
"""평균 342인데 확인해보니 건수가 기하급수적으로 늘어나서 너프하게 100 미만 살리기
65     7301
71     3655
29     2970
146    2856
4      2843"""
print(sum([i[1] for i in sorted_dict.items()]) / len(sorted_dict))

342.26143790849676


In [72]:
upper_100 = dict({i for i in sorted_dict.items() if i[1] >= 100})
under_100 = dict({i for i in sorted_dict.items() if i[1] < 100})

upper_100_temp = [[i[1]['g_num'], i[1]['u_num'], i[1]['material'], i[1]['speech'], i[1]['u'], i[1]['arg'], i[1]['arg_op1'], i[1]['arg_op2'], i[1]['label']] for i in df.iterrows() if i[1]['label'] in upper_100.keys()]
under_100_temp = [[i[1]['g_num'], i[1]['u_num'], i[1]['material'], i[1]['speech'], i[1]['u'], i[1]['arg'], i[1]['arg_op1'], i[1]['arg_op2'], i[1]['label']] for i in df.iterrows() if i[1]['label'] in under_100.keys()]

df_upper_100 = pd.DataFrame(upper_100_temp, columns=['g_num', 'u_num', 'material', 'speech', 'u', 'arg', 'arg_op1', 'arg_op2', 'label'])
df_under_100 = pd.DataFrame(under_100_temp, columns=['g_num', 'u_num', 'material', 'speech', 'u', 'arg', 'arg_op1', 'arg_op2', 'label'])

사용자 사전 - 실질 형태소 & 불변어만

In [75]:
morpheme = ['NNG', 'NNP', 'NNB', 'NR', 'NP', 'MM', 'MAG', 'MAJ', 'IC']
# morpheme = ['NNG', 'NNP', 'NNB', 'NR', 'NP', 'VV-R', 'VV-I', 'VA-R', 'VA-I', 'VX-R', 'VX-I', 'MM', 'MAG', 'MAJ', 'IC', 'XPN', 'XR']

In [76]:
user_dict = [[i[1]['lemma'], i[1]['formal'], i[1]['pos'], i[1]['sentiment'], i[1]['age'], i[1]['example'], i[1]['antonym'], i[1]['okt_pos'], i[1]['mecab_pos']] for i in user_dict_3.iterrows() if i[1]['mecab_pos'] in morpheme]
df_userdict = pd.DataFrame(user_dict, columns=['lemma', 'formal', 'pos', 'sentiment', 'age', 'example', 'antonym', 'okt_pos', 'mecab_pos'])

In [77]:
df_userdict.head()

,lemma,formal,pos,sentiment,age,example,antonym,okt_pos,mecab_pos
0,가장,가장,MAG,Neu,NaN,그게 가장 예쁘더라,NaN,Adverb,MAG
1,제일,가장,MAG,Neu,NaN,너가 제일 좋아,NaN,Adverb,MAG
2,젤,가장,MAG,Neu,MZ,젤 많이 먹어야지,NaN,Adverb,MAG
3,젤루,가장,MAG,Neu,MZ,난 귀신이 젤루 싫어,NaN,Adverb,MAG
4,최고로,가장,MAG,Neu,MZ,최고로 성능 좋지,NaN,Adverb,MAG


증강 작업

In [95]:
result = []

# 증강 할 유저 발화 51,007건 중 639건 대상
for k, i in tqdm(enumerate(df_under_100.iterrows())):
  # 원본
  result.append([i[1]['g_num'], i[1]['u_num'], i[1]['material'], i[1]['speech'], i[1]['u'], i[1]['arg'], i[1]['arg_op1'], i[1]['arg_op2'], None, i[1]['label']])

  input = [f"{i[0]}, {i[1]}" for i in kiwi.tokenize(i[1]['u'].replace(' ', '')) if i[1] in morpheme]

  # 사용자 사건에서 찾기
  for kk, ii in enumerate(df_userdict.iterrows()):
    udict = [f"{k[0]}, {k[1]}" for k in kiwi.tokenize(str(ii[1]['lemma']).replace(' ', '')) if k[1] in morpheme]
    check = sum([i in input for i in udict])
    temp = [input.index(i) for i in udict if i in input]

    # replacement
    if check == len(udict) and udict:  
      replace_set = list(set(df_userdict[df_userdict.formal == df_userdict.iloc[kk]['formal']].lemma.tolist()))
      if len(replace_set) > 1:
        # 교체될 단어가 문장의 첫 어절이면 스페이스를 뒤만.
        # 교체될 단어가 문장의 첫 어절이 아니면 스페이스를 앞뒤로.
        for k, v in enumerate(replace_set):
          replace_set_copy = replace_set.copy()
          replace_set_copy.pop(k)
          for r in replace_set_copy:
            # 어절이 맨 앞인 경우
            if str(i[1]['u']).find(v) == 0 and str(i[1]['u']) != str(i[1]['u']).replace(v+" ", r+" "):
              result.append([i[1]['g_num'], i[1]['u_num'], i[1]['material'], i[1]['speech'], str(i[1]['u']).replace(v+" ", r+" "), i[1]['arg'], i[1]['arg_op1'], i[1]['arg_op2'], r, i[1]['label']])
            # 어절이 중간에 있는 경우
            elif str(i[1]['u']).find(v) > 0 and str(i[1]['u']) != str(i[1]['u']).replace(" "+v+" ", " "+r+" "):
              result.append([i[1]['g_num'], i[1]['u_num'], i[1]['material'], i[1]['speech'], str(i[1]['u']).replace(" "+v+" ", " "+r+" "), i[1]['arg'], i[1]['arg_op1'], i[1]['arg_op2'], r, i[1]['label']])

639it [03:30,  3.04it/s]


In [96]:
df_under_aug = pd.DataFrame(result, columns=['g_num', 'u_num', 'material', 'speech', 'u', 'arg', 'arg_op1', 'arg_op2', 'user_dic', 'label'])

In [97]:
df_under_aug.drop_duplicates(inplace=True)
df_under_aug.reset_index(drop=True, inplace=True)

In [98]:
df_under_aug.label.value_counts()

23     1757
112     541
148     507
35      483
77      450
11      331
56      241
7       154
30      140
132     109
79       44
86       43
83       43
Name: label, dtype: int64

In [99]:
df_under_aug.shape

(4843, 10)

In [103]:
df_upper_100['user_dic'] = None
df_upper_100 = df_upper_100[['g_num', 'u_num', 'material', 'speech', 'u', 'arg', 'arg_op1', 'arg_op2', 'user_dic', 'label']]
df_temp = pd.concat([df_under_aug, df_upper_100])

In [104]:
df_temp.shape

(56570, 10)

In [116]:
df_temp.iloc[30:40]

,g_num,u_num,material,speech,u,arg,arg_op1,arg_op2,user_dic,label
30,77,3,경사,칭찬,결혼식 뷔페가 진짜 쩔었어,결혼식 뷔페,NaN,NaN,진짜,11
31,77,3,경사,칭찬,결혼식 뷔페가 넘넘 쩔었어,결혼식 뷔페,NaN,NaN,넘넘,11
32,77,3,경사,칭찬,결혼식 뷔페가 레알 쩔었어,결혼식 뷔페,NaN,NaN,레알,11
33,77,3,경사,칭찬,결혼식 뷔페가 베리 쩔었어,결혼식 뷔페,NaN,NaN,베리,11
34,77,3,경사,칭찬,결혼식 뷔페가 진심 쩔었어,결혼식 뷔페,NaN,NaN,진심,11
35,77,3,경사,칭찬,결혼식 뷔페가 렬루 쩔었어,결혼식 뷔페,NaN,NaN,렬루,11
36,77,3,경사,칭찬,결혼식 뷔페가 쏘 쩔었어,결혼식 뷔페,NaN,NaN,쏘,11
37,77,3,경사,칭찬,결혼식 뷔페가 정말 쩔었어,결혼식 뷔페,NaN,NaN,정말,11
38,77,3,경사,칭찬,결혼식 뷔페가 넘 쩔었어,결혼식 뷔페,NaN,NaN,넘,11
39,77,3,경사,칭찬,결혼식 뷔페가 너무나 쩔었어,결혼식 뷔페,NaN,NaN,너무나,11


### Mecab

test<br>
사용자 사전 등록이 안되는데 원인 파악 안 됨.

In [ ]:
from konlpy.tag import Mecab

m = Mecab()

In [ ]:
print(f"겟또하다 Verb : {m.pos('핸드폰을 겟또하다.')}")
print(f"득템하다 Verb : {m.pos('동전을 득템하다.')}")
print(f"가상화폐 Noun : {m.pos('가상화폐를 살거야')}")
print(f"틀니딱딱 Noun : {m.pos('틀니딱딱 거리지마~')}")
print(f"젤루 Adverb : {m.pos('엄마가 젤루 좋아')}")
print(f"틀림없이 Adverb : {m.pos('이번엔 틀림없이 합격할 거야')}")
print(f"쏘 러블리하다 Adjective : {m.pos('쏘 러블리하다')}")

겟또하다 Verb : [('핸드폰', 'NNG'), ('을', 'JKO'), ('겟', 'NNP'), ('또', 'MAG'), ('하', 'VV'), ('다', 'EF'), ('.', 'SF')]
득템하다 Verb : [('동전', 'NNG'), ('을', 'JKO'), ('득템', 'NNG'), ('하', 'XSV'), ('다', 'EF'), ('.', 'SF')]
가상화폐 Noun : [('가상', 'NNG'), ('화폐', 'NNG'), ('를', 'JKO'), ('살', 'VV+ETM'), ('거', 'NNB'), ('야', 'VCP+EF')]
틀니딱딱 Noun : [('틀니', 'NNG'), ('딱딱', 'MAG'), ('거리', 'NNG'), ('지마', 'NNG'), ('~', 'SY')]
젤루 Adverb : [('엄마', 'NNG'), ('가', 'JKS'), ('젤', 'NNG'), ('루', 'JKB'), ('좋', 'VA'), ('아', 'EC')]
틀림없이 Adverb : [('이번', 'NNG'), ('엔', 'JKB+JX'), ('틀림없이', 'MAG'), ('합격', 'NNG'), ('할', 'XSV+ETM'), ('거', 'NNB'), ('야', 'VCP+EF')]
쏘 러블리하다 Adjective : [('쏘', 'NNP'), ('러블리', 'NNP'), ('하', 'VV'), ('다', 'EC')]


Add Dict

In [ ]:
!pip3 install jamo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from jamo import h2j, j2hcj

def get_jongsung_TF(sample_text):
    sample_text_list = list(sample_text)
    last_word = sample_text_list[-1]
    last_word_jamo_list = list(j2hcj(h2j(last_word)))
    last_jamo = last_word_jamo_list[-1]

    jongsung_TF = "T"

    if last_jamo in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅘ', 'ㅚ', 'ㅙ', 'ㅝ', 'ㅞ', 'ㅢ', 'ㅐ,ㅔ', 'ㅟ', 'ㅖ', 'ㅒ']:
        jongsung_TF = "F"

    return jongsung_TF

In [ ]:
paths = {
    'NNG.csv':'NNG',
    'NNP.csv':'NNP',
    'NR.csv':'NR',
    'NP.csv':'NP',
    'VV.csv':'VV',
    'VA.csv':'VA',
    'MM.csv':'MM',
    'MAG.csv':'MAG',
    'MAJ.csv':'MAJ',
    'IC.csv':'IC',
    'J.csv':'JO',
}

In [ ]:
for p in paths:
  with open(f'/tmp/mecab-ko-dic-2.1.1-20180720/{p}', "r", encoding='utf-8') as f:
    user_dict = f.readlines()

  for i in user_dict_3[user_dict_3.mecab_pos == paths[p]].iterrows():
    jongsung_TF = get_jongsung_TF(str(i[1]['lemma']))
    line = '{},*,*,*,{},*,{},{},*,*,*,*,*\n'.format(i[1]['lemma'], paths[p], jongsung_TF, i[1]['lemma'])
    user_dict.append(line)
  
  with open(f'/tmp/mecab-ko-dic-2.1.1-20180720/{p}', 'w', encoding='utf-8') as f:
    for line in user_dict:
      f.write(line)

  user_dict = []

In [ ]:
!bash /tmp/mecab-ko-dic-2.1.1-20180720/tools/add-userdic.sh
!make /tmp/mecab-ko-dic-2.1.1-20180720/install

generating userdic...
nnp.csv
/tmp/mecab-ko-dic-2.1.1-20180720/tools/../model.def is not a binary model. reopen it as text mode...
reading /tmp/mecab-ko-dic-2.1.1-20180720/tools/../user-dic/nnp.csv ... 
done!
person.csv
/tmp/mecab-ko-dic-2.1.1-20180720/tools/../model.def is not a binary model. reopen it as text mode...
reading /tmp/mecab-ko-dic-2.1.1-20180720/tools/../user-dic/person.csv ... 
done!
place.csv
/tmp/mecab-ko-dic-2.1.1-20180720/tools/../model.def is not a binary model. reopen it as text mode...
reading /tmp/mecab-ko-dic-2.1.1-20180720/tools/../user-dic/place.csv ... 
done!
test -z "model.bin matrix.bin char.bin sys.dic unk.dic" || rm -f model.bin matrix.bin char.bin sys.dic unk.dic
/usr/local/libexec/mecab/mecab-dict-index -d . -o . -f UTF-8 -t UTF-8
reading ./unk.def ... 13
emitting double-array: 100% |###########################################| 
reading ./NNP.csv ... 2453
reading ./Group.csv ... 3176
reading ./Inflect.csv ... 44820
reading ./VV.csv ... 8729
reading ./NR

check

In [ ]:
with open("/tmp/mecab-ko-dic-2.1.1-20180720/VV.csv", 'r', encoding='utf-8') as f:
  file_new = f.readlines()
file_new[-20:]

['속 썩이다,*,*,*,VV,*,F,속 썩이다,*,*,*,*,*\n',
 '시전하다,*,*,*,VV,*,F,시전하다,*,*,*,*,*\n',
 '애프터 신청하다,*,*,*,VV,*,F,애프터 신청하다,*,*,*,*,*\n',
 '궁예질하다,*,*,*,VV,*,F,궁예질하다,*,*,*,*,*\n',
 '갑분싸되다,*,*,*,VV,*,F,갑분싸되다,*,*,*,*,*\n',
 '급 어색해지다,*,*,*,VV,*,F,급 어색해지다,*,*,*,*,*\n',
 '갑자기 분위기 싸해지다,*,*,*,VV,*,F,갑자기 분위기 싸해지다,*,*,*,*,*\n',
 '고스팅하다,*,*,*,VV,*,F,고스팅하다,*,*,*,*,*\n',
 '잠수 타다,*,*,*,VV,*,F,잠수 타다,*,*,*,*,*\n',
 '꾸꾸꾸,*,*,*,VV,*,F,꾸꾸꾸,*,*,*,*,*\n',
 '꾸미고 꾸미고 꾸몄다,*,*,*,VV,*,F,꾸미고 꾸미고 꾸몄다,*,*,*,*,*\n',
 '발컨하다,*,*,*,VV,*,F,발컨하다,*,*,*,*,*\n',
 '발로 컨트롤하다,*,*,*,VV,*,F,발로 컨트롤하다,*,*,*,*,*\n',
 '오저치고,*,*,*,VV,*,F,오저치고,*,*,*,*,*\n',
 '오늘 저녁에 치킨 고,*,*,*,VV,*,F,오늘 저녁에 치킨 고,*,*,*,*,*\n',
 '이생망,*,*,*,VV,*,T,이생망,*,*,*,*,*\n',
 '이번 생은 망했다,*,*,*,VV,*,F,이번 생은 망했다,*,*,*,*,*\n',
 '혼틈,*,*,*,VV,*,T,혼틈,*,*,*,*,*\n',
 '혼란한 틈 타,*,*,*,VV,*,F,혼란한 틈 타,*,*,*,*,*\n',
 '일며들다,*,*,*,VV,*,F,일며들다,*,*,*,*,*\n']

In [ ]:
from konlpy.tag import Mecab

m = Mecab(dicpath='/tmp/mecab-ko-dic-2.1.1-20180720')
# m = Mecab()

In [ ]:
print(f"겟또하다 Verb : {m.pos('핸드폰을 겟또하다. 쿠쿠루삥뽕')}")
print(f"득템하다 Verb : {m.pos('동전을 득템하다.')}")
print(f"가상화폐 Noun : {m.pos('가상화폐를 살거야')}")
print(f"틀니딱딱 Noun : {m.pos('틀니딱딱 거리지마~')}")
print(f"젤루 Adverb : {m.pos('엄마가 젤루 좋아')}")
print(f"틀림없이 Adverb : {m.pos('이번엔 틀림없이 합격할 거야')}")
print(f"쏘 러블리하다 Adjective : {m.pos('쏘 러블리하다')}")

겟또하다 Verb : [('핸드폰', 'NNG'), ('을', 'JKO'), ('겟또', 'NNG'), ('하', 'VV'), ('다', 'EF'), ('.', 'SF'), ('쿠쿠루삥뽕', 'IC')]
득템하다 Verb : [('동전', 'NNG'), ('을', 'JKO'), ('득템', 'NNG'), ('하', 'VV'), ('다', 'EF'), ('.', 'SF')]
가상화폐 Noun : [('가상화폐', 'NNG'), ('를', 'JKO'), ('살', 'VV+ETM'), ('거', 'NNB'), ('야', 'VCP+EF')]
틀니딱딱 Noun : [('틀니딱딱', 'NNG'), ('거리', 'NNG'), ('지마', 'NNG'), ('~', 'SY')]
젤루 Adverb : [('엄마', 'NNG'), ('가', 'JKS'), ('젤루', 'MAG'), ('좋', 'VA'), ('아', 'EC')]
틀림없이 Adverb : [('이번', 'NNG'), ('엔', 'JKB+JX'), ('틀림없이', 'MAG'), ('합', 'NNG'), ('격할', 'VV+ETM'), ('거', 'NNB'), ('야', 'VCP+EF')]
쏘 러블리하다 Adjective : [('쏘', 'MAG'), ('러블리', 'NNG'), ('하', 'VV'), ('다', 'EC')]


### Customized KoNLPy

test
사용자 사전 등록이 안되는데 원인 파악 안 됨.

In [ ]:
!pip install customized_konlpy

In [ ]:
from ckonlpy.tag import Twitter

In [ ]:
twitter = Twitter()

In [ ]:
tags = ['Noun', 'Verb', 'Adjective', 'Adverb', 'Exclamation', 'Eomi', 'Determiner']

In [ ]:
for t in tags:
  for i in user_dict_3[user_dict_3.okt_pos==t].iterrows():
    twitter.add_dictionary(str(i[1]['lemma']), t)

check

In [ ]:
from ckonlpy.tag import Twitter

In [ ]:
twitter = Twitter()

In [ ]:
print(f"겟또하다 Verb : {twitter.pos('핸드폰을 겟또하다.')}")
print(f"득템하다 Verb : {twitter.pos('동전을 득템하다.')}")
print(f"가상화폐 Noun : {twitter.pos('가상화폐를 살거야')}")
print(f"틀니딱딱 Noun : {twitter.pos('틀니딱딱 거리지마~')}")
print(f"젤루 Adverb : {twitter.pos('엄마가 젤루 좋아')}")
print(f"틀림없이 Adverb : {twitter.pos('이번엔 틀림없이 합격할 거야')}")
print(f"쏘 러블리하다 Adjective : {twitter.pos('쏘 러블리하다')}")